# 1D: Linha Reta


In [9]:
import numpy as np
import plotly.graph_objs as go
import random
from itertools import combinations


# Função para calcular a distância de Hamming
def hamming_distance(a, b):
    return sum(el1 != el2 for el1, el2 in zip(a, b))


# Função para projetar nD para 3D
def project_nd_to_3d(v):
    v = np.array(v)
    return tuple(v[:3] + 0.5 * v[3] if len(v) > 3 else np.pad(v, (0, 3 - len(v))))


# Função para aplicar transformação matemática
def apply_transformation(vertices, func):
    return [func(np.array(v)) for v in vertices]


# Função para gerar pontos e arestas de uma espiral em 4D
def generate_4d_spiral_points_and_edges(num_points):
    t = np.linspace(0, 4 * np.pi, num_points)

    x = np.sin(t)
    y = np.cos(t)
    z = t
    w = np.sin(2 * t)

    vertices = list(zip(x, y, z, w))
    transformed_vertices = apply_transformation(vertices, lambda v: tuple(v))
    projected_vertices = [project_nd_to_3d(v) for v in transformed_vertices]
    x, y, z = zip(*projected_vertices)

    edges = [
        (projected_vertices[i], projected_vertices[i + 1])
        for i in range(num_points - 1)
    ]

    return vertices, projected_vertices, edges


# Exemplo de função de transformação (identidade)
def identity_function(v):
    return v


# Função para calcular a distância euclidiana entre dois pontos
def euclidean_distance(p1, p2):
    return np.sqrt(np.sum((np.array(p1) - np.array(p2)) ** 2))


# Função para simular um caminho aleatório usando Monte Carlo
def monte_carlo_path(vertices, edges, start, end, iterations=1000):
    best_path = None
    best_distance = float("inf")

    for _ in range(iterations):
        current_path = [start]
        current_distance = 0
        current_point = start

        while not np.array_equal(current_point, end):
            next_points = [
                edge[1]
                for edge in edges
                if np.array_equal(edge[0], current_point)
                and edge[1] not in current_path
            ]
            if not next_points:
                break
            next_point = random.choice(next_points)
            current_path.append(next_point)
            current_distance += euclidean_distance(current_point, next_point)
            current_point = next_point

        if np.array_equal(current_point, end) and current_distance < best_distance:
            best_path = current_path
            best_distance = current_distance

    return best_path, best_distance


# Função principal para visualizar a função matemática
def visualize_points_and_edges(x, y, z, edges, best_path=None):
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in edges:
        x0, y0, z0 = edge[0]
        x1, y1, z1 = edge[1]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    fig = go.Figure(
        data=[
            go.Scatter3d(
                x=edge_x,
                y=edge_y,
                z=edge_z,
                mode="lines",
                line=dict(color="blue", width=2),
            ),
            go.Scatter3d(
                x=x, y=y, z=z, mode="markers", marker=dict(size=4, color="red")
            ),
        ],
        layout=go.Layout(
            scene=dict(xaxis_title="X", yaxis_title="Y", zaxis_title="Z"),
            width=700,
            margin=dict(r=20, b=10, l=10, t=10),
        ),
    )

    if best_path:
        best_x, best_y, best_z = zip(*best_path)
        fig.add_trace(
            go.Scatter3d(
                x=best_x,
                y=best_y,
                z=best_z,
                mode="lines+markers",
                line=dict(color="green", width=4),
                marker=dict(size=4, color="green"),
            )
        )

    fig.show()


In [10]:
# Função para gerar pontos e arestas de uma linha reta em 1D
def generate_line_points_and_edges_1d(num_points, start=-1, end=1):
    x = np.linspace(start, end, num_points)
    y = np.zeros(num_points)
    z = np.zeros(num_points)

    vertices = list(zip(x, y, z))
    edges = [(vertices[i], vertices[i + 1]) for i in range(num_points - 1)]

    return vertices, vertices, edges


# Gerar pontos e arestas da linha reta em 1D
vertices_line_1d, projected_line_1d, edges_line_1d = generate_line_points_and_edges_1d(
    100
)
start_point_1d = projected_line_1d[0]
end_point_1d = projected_line_1d[-1]
best_path_1d, best_distance_1d = monte_carlo_path(
    projected_line_1d, edges_line_1d, start_point_1d, end_point_1d, iterations=1000
)
x_line_1d, y_line_1d, z_line_1d = zip(*projected_line_1d)
visualize_points_and_edges(x_line_1d, y_line_1d, z_line_1d, edges_line_1d, best_path_1d)


# 2D: Circunferência

In [11]:
# Função para gerar pontos e arestas de uma circunferência em 2D
def generate_circle_points_and_edges_2d(num_points, radius=1):
    theta = np.linspace(0, 2 * np.pi, num_points)

    x = radius * np.cos(theta)
    y = radius * np.sin(theta)
    z = np.zeros(num_points)

    vertices = list(zip(x, y, z))
    edges = [(vertices[i], vertices[(i + 1) % num_points]) for i in range(num_points)]

    return vertices, vertices, edges


# Gerar pontos e arestas da circunferência em 2D
vertices_circle_2d, projected_circle_2d, edges_circle_2d = (
    generate_circle_points_and_edges_2d(100)
)
start_point_2d = projected_circle_2d[0]
end_point_2d = projected_circle_2d[50]  # Aproximadamente o ponto oposto
best_path_2d, best_distance_2d = monte_carlo_path(
    projected_circle_2d, edges_circle_2d, start_point_2d, end_point_2d, iterations=1000
)
x_circle_2d, y_circle_2d, z_circle_2d = zip(*projected_circle_2d)
visualize_points_and_edges(
    x_circle_2d, y_circle_2d, z_circle_2d, edges_circle_2d, best_path_2d
)


# 3D: Hélice

In [12]:
# Função para gerar pontos e arestas de uma hélice em 3D
def generate_helix_points_and_edges_3d(num_points, a=1, b=0.1):
    t = np.linspace(0, 4 * np.pi, num_points)

    x = a * np.cos(t)
    y = a * np.sin(t)
    z = b * t

    vertices = list(zip(x, y, z))
    edges = [(vertices[i], vertices[i + 1]) for i in range(num_points - 1)]

    return vertices, vertices, edges


# Gerar pontos e arestas da hélice em 3D
vertices_helix_3d, projected_helix_3d, edges_helix_3d = (
    generate_helix_points_and_edges_3d(100)
)
start_point_3d = projected_helix_3d[0]
end_point_3d = projected_helix_3d[-1]
best_path_3d, best_distance_3d = monte_carlo_path(
    projected_helix_3d, edges_helix_3d, start_point_3d, end_point_3d, iterations=1000
)
x_helix_3d, y_helix_3d, z_helix_3d = zip(*projected_helix_3d)
visualize_points_and_edges(
    x_helix_3d, y_helix_3d, z_helix_3d, edges_helix_3d, best_path_3d
)


# 3D: Esfera

In [13]:
# Função para gerar pontos e arestas de uma esfera em 3D
def generate_sphere_points_and_edges_3d(num_points):
    phi = np.linspace(0, 2 * np.pi, num_points)
    theta = np.linspace(0, np.pi, num_points)
    phi, theta = np.meshgrid(phi, theta)

    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)

    vertices = list(zip(x.ravel(), y.ravel(), z.ravel()))

    edges = []
    for i in range(num_points):
        for j in range(num_points):
            if j + 1 < num_points:
                edges.append(
                    (vertices[i * num_points + j], vertices[i * num_points + j + 1])
                )
            if i + 1 < num_points:
                edges.append(
                    (vertices[i * num_points + j], vertices[(i + 1) * num_points + j])
                )

    return vertices, vertices, edges


# Gerar pontos e arestas da esfera em 3D
vertices_sphere_3d, projected_sphere_3d, edges_sphere_3d = (
    generate_sphere_points_and_edges_3d(30)
)
start_point_3d = projected_sphere_3d[0]
end_point_3d = projected_sphere_3d[-1]

# Encontrar o caminho mais curto usando Monte Carlo
best_path_3d, best_distance_3d = monte_carlo_path(
    projected_sphere_3d, edges_sphere_3d, start_point_3d, end_point_3d, iterations=1000
)

# Visualizar pontos e arestas da esfera em 3D com o caminho mais curto
x_sphere_3d, y_sphere_3d, z_sphere_3d = zip(*projected_sphere_3d)
visualize_points_and_edges(
    x_sphere_3d, y_sphere_3d, z_sphere_3d, edges_sphere_3d, best_path_3d
)


# 4D: Hipercubo

In [14]:
# Função para gerar os vértices do hipercubo de n dimensões
def generate_hypercube_vertices(n):
    return [tuple([int(x) for x in format(i, f"0{n}b")]) for i in range(2**n)]


# Função para gerar pontos e arestas de um hipercubo em 4D
def generate_hypercube_points_and_edges_4d():
    vertices = generate_hypercube_vertices(4)
    transformed_vertices = apply_transformation(vertices, lambda v: tuple(v))
    projected_vertices = [project_nd_to_3d(v) for v in transformed_vertices]
    x, y, z = zip(*projected_vertices)

    edges = [
        (projected_vertices[i], projected_vertices[j])
        for i, j in combinations(range(len(vertices)), 2)
        if hamming_distance(vertices[i], vertices[j]) == 1
    ]

    return vertices, projected_vertices, edges


# Gerar pontos e arestas do hipercubo em 4D
vertices_hypercube_4d, projected_hypercube_4d, edges_hypercube_4d = (
    generate_hypercube_points_and_edges_4d()
)
start_point_4d = projected_hypercube_4d[0]
end_point_4d = projected_hypercube_4d[-1]
best_path_4d, best_distance_4d = monte_carlo_path(
    projected_hypercube_4d,
    edges_hypercube_4d,
    start_point_4d,
    end_point_4d,
    iterations=1000,
)
x_hypercube_4d, y_hypercube_4d, z_hypercube_4d = zip(*projected_hypercube_4d)
visualize_points_and_edges(
    x_hypercube_4d, y_hypercube_4d, z_hypercube_4d, edges_hypercube_4d, best_path_4d
)
